 #сделать таблицу выполненного комплекса по скважине. или скважинам? нудно выделить то, что есть в ласах и то что было занесено в lqc

In [137]:
def imports_dicts():

    with open('famdict.json', 'r', encoding='utf-8') as f:
        famdict = json.load(f)
    with open('lqcdict.json', 'r', encoding='utf-8') as f:
        lqcdict = json.load(f)
    return famdict, lqcdict

In [138]:
def get_folder_path_and_splice_state():
    root = tk.Tk()
    root.geometry("400x150")
    root.title("Folder path and Splice state form")

    folder_path = ""

    def select_folder_path():
        nonlocal folder_path
        folder_path = filedialog.askdirectory()
        root.destroy()

    def get_splice_state_and_close():
        splice_state = splice_checkbutton_var.get()
        root.destroy()

    folder_path_label = tk.Label(root, text="Select Folder:")
    folder_path_label.pack()
    folder_path_entry = tk.Entry(root, width=50)
    folder_path_entry.pack()

    select_folder_path_button = tk.Button(root, text="Browse", command=select_folder_path)
    select_folder_path_button.pack()

    splice_checkbutton_var = tk.IntVar()
    splice_checkbutton = tk.Checkbutton(root, text="Splice", variable=splice_checkbutton_var)
    splice_checkbutton.pack()

    ok_button = tk.Button(root, text="Ok", command=get_splice_state_and_close)
    ok_button.pack()

    root.mainloop()

    return folder_path, splice_checkbutton_var.get()

In [139]:
def datasetlist_sort(dbdatasetlist, class_distance=300):
    dsets = []

    for dset in dbdatasetlist:
        top = dset[1]
        bottom = dset[2]
        if top > bottom:
            top, bottom = bottom, top
        dsets.append([dset[0], top, bottom, bottom - top])
    ds_df = pd.DataFrame(dsets, columns=['project_index', 'top', 'bottom', 'size'])
    ds_df.sort_values(by=['top', 'size'], inplace=True)

    dbs = DBSCAN(eps=class_distance, min_samples=1).fit(ds_df[['top', 'bottom']])
    ds_df['labels'] = dbs.labels_

    print(ds_df.to_string())

    return ds_df[['project_index', 'labels']].values.tolist()

In [140]:

#идея в том чтобы возвращать массив данных кривойб юниты.чтобы сразу записать в обьект welly
#нужно найти кривую проверить юниты и если не совпадают то пересчитать в нужные записать все в массив и вернуть его
def lqclogdata(wLQC, dataset_ind, dslabel_):
    logs = []
    lqcname = ''
    unitname = ''
    for cname in las_project[dataset_ind].data.keys():
        if any(cname in sublist for sublist in list(lqcdict.values())):  # есть ли кривая в словаре lqcdict гделибо
            lqcname = list(lqcdict.keys())[list(lqcdict.values()).index(
                [sublist for sublist in list(lqcdict.values()) if cname in sublist][
                    0])] + dslabel_  #выбираем из словаря ключ соотвествующий найденному в значениях имени кривой
            if lqcname in wLQC.data.keys():
                clist = [i for i in wLQC.data.keys() if i.find(lqcname) != -1]
                if len(clist) >= 1:
                    lqcname = lqcname + "_" + str(len(clist))

            if las_project[dataset_ind].data[cname].units != famdict[lqcname][1]:

                if lqcname == "NPHI":  # добавить проверку по конкретным методам ГК НК
                    if las_project[dataset_ind].data[cname].mean().iloc[0] < 1:
                        unitname = "v/v"
                    else:
                        unitname = "%"

                elif lqcname == "GR":
                    if las_project[dataset_ind].data[cname].mean().iloc[0] < 50:
                        unitname = "uR/H"
                    else:
                        unitname = "Gapi"

                elif lqcname == "CALI":
                    if las_project[dataset_ind].data[cname].mean().iloc[0] < 1:
                        unitname = "m"
                    else:
                        unitname = "mm"
                else:
                    unitname = famdict[lqcname][1]
            else:
                unitname = las_project[dataset_ind].data[cname].units

            print(lqcname, unitname)
            logs.append([cname, lqcname, unitname])


    return logs

In [141]:
#find list of las files in folder las_path including subfolders
def make_las_project(las_path):
    las_list = []
    for root, dirs, files in os.walk(las_path):
        for file in files:
            if file.lower().endswith(".las"):
                las_list.append(os.path.join(root, file))

    las_project = Project.from_las(las_list, index='M')

    return las_project

In [142]:
def make_table():
    import xlwt

    # Load the LAS file into a well object
    well = welly.Well.from_las('your_file.las')

    # Create a new workbook and worksheet for the output
    workbook = xlwt.Workbook()
    worksheet = workbook.add_sheet('Logs')

    # Write the header row
    worksheet.write(0, 0, 'Log Name')
    worksheet.write(0, 1, 'Start Depth')
    worksheet.write(0, 2, 'Stop Depth')

    # Loop through all the logs in the well object
    for log in well.logs:
        # Get the log name
        log_name = log.name

        # Get the start and stop depths
        start_depth = log.start
        stop_depth = log.stop

        # Write the log name and depths to the worksheet
        row = worksheet.last_used_row + 1
        worksheet.write(row, 0, log_name)
        worksheet.write(row, 1, start_depth)
        worksheet.write(row, 2, stop_depth)

    # Save the workbook to a file
    workbook.save('logs.xls')


In [143]:
def make_intevals(las_project):
    wells_list = {}
    for ind, w in enumerate(las_project):

        well_name = str(w.name).replace(" ", "")
        ds_start = w.header[w.header['mnemonic'] == 'STRT']['value'].values[0]
        ds_stop = w.header[w.header['mnemonic'] == 'STOP']['value'].values[0]

        if well_name == 'WELL' or well_name == '':
            well_name = str(w.fname)
            print("!!! file {} does not contain well name. Using file name instead".format(w.fname)) #задавать вручную

        if well_name not in wells_list.keys() :
            wells_list[well_name] = []

        wells_list[well_name].append([ind, ds_start, ds_stop])

    print(wells_list)
    return wells_list

In [144]:
def make_lqc_las(wells_list, las_project):
    for well in wells_list:
        print(well)
        #determine maximum depth for lqc dataset and create list for dataframe index

        max_depth = max([w[2] for w in wells_list[well]])
        min_depth = min([w[1] for w in wells_list[well]])
        #MD = [round(d * 0.1, 1) for d in range(0, int(round((max_depth + 5) / 0.1, 0)) + 1)]

        wLQC = Well()
        wLQC.name = well



        for dataset_ind, dslabel in datasetlist_sort(wells_list[well],300):
            if dslabel == 0:
                dslabel_ = ""
            else:
                dslabel_ = "_" + str(dslabel)


            #copy curve object correctly to preserve unit information
            #assign or check units for curves
            for cname, lqcname, unitname in lqclogdata(wLQC, dataset_ind, dslabel_):

                wLQC.data[lqcname] = Curve(data=las_project[dataset_ind].data[cname].values, index=las_project[dataset_ind].data[cname].index,mnemonic=lqcname, units=unitname)\
                    .to_basis(start=min_depth-5, stop=max_depth + 5, step=0.1)

        if len(wLQC.data.keys()) > 0:
            wLQC.to_las(well +"_LQC.las")

In [145]:
import json
import pandas as pd
from sklearn.cluster import DBSCAN
import ctypes
from welly import Well, Project, Curve
import matplotlib.pyplot as plt
import os
import tkinter as tk
from tkinter import filedialog

if __name__ == "__main__":
    famdict, lqcdict = imports_dicts()

    #las_path, LWD_splice = folder_path_and_splice_state_form()
    las_path = "/Users/victorpotysyev/PycharmProjects/lqc_maker/las_data/1607"

    las_project = make_las_project(las_path)

    wells_list = make_intevals(las_project)
    make_lqc_las(wells_list, las_project)




0it [00:00, ?it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
1it [00:00,  5.48it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
2it [00:00,  7.31it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
3it [00:00,  5.85it/s]/opt/anaconda3/lib/python3.9/site-packages/welly/las.py:144: UserWarning: Warning, LAS version 3.0 not yet supported. Attempting to use LAS 1.2 and 2.0 parsing logic for LAS 3.0.
  datasets = from_lasio(las)
4it [00:00,  5.77it/s]


{'1607': [[0, 2124, 3455], [1, 2025, 3447.8], [2, 2025, 3456.1], [3, 2025, 3462.6]]}
1607
   project_index   top  bottom    size  labels
1              1  2025  3447.8  1422.8       0
2              2  2025  3456.1  1431.1       0
3              3  2025  3462.6  1437.6       0
0              0  2124  3455.0  1331.0       0
DTP us/m
IL1 Ohmm
IL2 Ohmm
IL3 Ohmm
IL4 Ohmm
IL5 Ohmm
RT Ohmm
CALI mm
NKTB UE
GK uR/h
RHOB g/cm3
NPHI %
